In [1]:
import cupy

print(cupy.show_config())


OS                           : Linux-6.14.0-29-generic-x86_64-with-glibc2.39
Python Version               : 3.12.3
CuPy Version                 : 12.3.0
CuPy Platform                : NVIDIA CUDA
NumPy Version                : 1.26.4
SciPy Version                : None
Cython Build Version         : 0.29.36
Cython Runtime Version       : None
CUDA Root                    : /usr/local/cuda
nvcc PATH                    : /usr/local/cuda/bin/nvcc
CUDA Build Version           : 12020
CUDA Driver Version          : 12090
CUDA Runtime Version         : 12090
cuBLAS Version               : (available)
cuFFT Version                : 11401
cuRAND Version               : 10310
cuSOLVER Version             : (11, 7, 5)
cuSPARSE Version             : (available)
NVRTC Version                : (12, 9)
Thrust Version               : 200101
CUB Build Version            : 200101
Jitify Build Version         : <unknown>
cuDNN Build Version          : (not loaded; try `import cupy.cuda.cudnn` first)
cuD

In [2]:
# ========================
# Cell 1: 基础检查
# ========================
import spacy
from pathlib import Path
import os

# 查看 spaCy 版本和可用组件
spacy.info()


{'spacy_version': '3.8.7',
 'location': '/home/jesse/Projects/myenvs/spacy_lv/lib/python3.12/site-packages/spacy',
 'platform': 'Linux-6.14.0-29-generic-x86_64-with-glibc2.39',
 'python_version': '3.12.3',
 'pipelines': {'lv_xlmr_base_without_lemma': '1.0.0',
  'lv_spacy_without_lemma': '1.0.0',
  'lv_bert_without_lemma': '1.0.0'}}

In [3]:
# ========================
# Cell 2: 创建 corpus 和 models 文件夹
# ========================
Path("./corpus").mkdir(parents=True, exist_ok=True)

# 原始训练结果和子目录
Path("./models").mkdir(parents=True, exist_ok=True)
Path("./models/spacy_lv").mkdir(parents=True, exist_ok=True)
Path("./models/bert_lv").mkdir(parents=True, exist_ok=True)

# 打包后的模型
Path("./packages").mkdir(parents=True, exist_ok=True)

# config文件路径
Path("./config").mkdir(parents=True, exist_ok=True)

print("✅ 基础目录结构已创建完成")


✅ 基础目录结构已创建完成


In [4]:
# ========================
# Cell 3: 转换 conllu 到 spaCy 格式
# ========================
!python -m spacy convert ud_latvian/lv_lvtb-ud-train.conllu ./corpus -n 10
!python -m spacy convert ud_latvian/lv_lvtb-ud-dev.conllu ./corpus -n 10
!python -m spacy convert ud_latvian/lv_lvtb-ud-test.conllu ./corpus -n 10


ℹ Grouping every 10 sentences into a document.
✔ Generated output file (1506 documents):
corpus/lv_lvtb-ud-train.spacy
ℹ Grouping every 10 sentences into a document.
✔ Generated output file (208 documents):
corpus/lv_lvtb-ud-dev.spacy
ℹ Grouping every 10 sentences into a document.
✔ Generated output file (240 documents):
corpus/lv_lvtb-ud-test.spacy


In [5]:
# ========================
# Cell 4: 初始化 config
# ========================
!python -m spacy init config ./config/config_spacy.cfg --lang lv --pipeline tok2vec,tagger,morphologizer,parser,senter --optimize efficiency


ℹ Generated config template specific for your use case
- Language: lv
- Pipeline: tagger, morphologizer, parser, senter
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config/config_spacy.cfg
You can now add your data and train your pipeline:
python -m spacy train config_spacy.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [6]:
# ========================
# Cell 5: 修改 config 数据路径
# ========================
cfg_text = Path("./config/config_spacy.cfg").read_text(encoding="utf-8")
cfg_text = cfg_text.replace("train = null", "train = ./corpus/lv_lvtb-ud-train.spacy")
cfg_text = cfg_text.replace("dev = null", "dev = ./corpus/lv_lvtb-ud-dev.spacy")
Path("./config/config_spacy.cfg").write_text(cfg_text, encoding="utf-8")
print("✅ 已经修改 config_spacy.cfg 的数据路径")


✅ 已经修改 config_spacy.cfg 的数据路径


In [8]:
# ========================
# Cell 6: 使用 GPU 训练（没有 GPU 就去掉 --gpu-id 0）
# ========================
!python -m spacy train ./config/config_spacy.cfg --output ./models/spacy_lv --paths.train ./corpus/lv_lvtb-ud-train.spacy --paths.dev ./corpus/lv_lvtb-ud-dev.spacy --gpu-id 0


ℹ Saving to output directory: models/spacy_lv
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'tagger', 'morphologizer', 'parser',
'senter']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS TAGGER  LOSS MORPH...  LOSS PARSER  LOSS SENTER  TAG_ACC  POS_ACC  MORPH_ACC  DEP_UAS  DEP_LAS  SENTS_P  SENTS_R  SENTS_F  SCORE 
---  ------  ------------  -----------  -------------  -----------  -----------  -------  -------  ---------  -------  -------  -------  -------  -------  ------
  0       0          0.00       202.94         202.86       447.71       112.50    18.53    31.64      21.10     8.67     6.82     0.01     0.19     0.02    0.13
  0     200       3295.73     23142.84       21814.87     33271.65      2844.51    51.99    78.04      58.33    49.78    32.90    23.18    41.30    29.70    0.48
  0     400

In [9]:
# ========================
# Cell 7: 模型评估
# ========================
!python -m spacy evaluate ./models/spacy_lv/model-best ./corpus/lv_lvtb-ud-test.spacy --gpu-id 0


ℹ Using GPU: 0

================================== Results ==================================

TOK      99.53
TAG      86.64
POS      94.83
MORPH    90.76
UAS      80.58
LAS      74.95
SENT P   97.10
SENT R   96.49
SENT F   96.80
SPEED    23889


============================== MORPH (per feat) ==============================

                P        R        F
ExtPos      86.47    82.73    84.56
Case        92.33    91.76    92.05
Gender      93.65    92.95    93.30
Number      94.18    92.99    93.58
Person      94.86    93.86    94.36
PronType    98.53    97.63    98.08
Evident     94.16    93.35    93.76
Mood        93.30    92.25    92.77
Polarity    94.06    93.10    93.58
Tense       91.17    89.48    90.32
VerbForm    92.78    91.81    92.29
Voice       93.75    91.85    92.79
Definite    90.00    88.04    89.01
Degree      92.50    90.57    91.52
Poss       100.00   100.00   100.00
NumType     98.65    76.91    86.44
Reflex      92.86    93.66    93.25
Aspect      89.77    85.4

In [10]:
# ========================
# Cell 8: 不加入 Stanza lemmatizer，直接保存模型
# ========================
import spacy

# 加载训练好的模型
nlp = spacy.load("./models/spacy_lv/model-best")

# 保存到新路径（可选）
nlp.to_disk("./models/spacy_lv/model-no-lemma")
print("✅ 模型已保存到 ./models/spacy_lv/model-no-lemma")



/home/jesse/Projects/myenvs/spacy_lv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ 模型已保存到 ./models/spacy_lv/model-no-lemma


In [11]:
# ========================
# Cell 8.5: Notebook 打包修正版（使用相对路径）
# ========================
from spacy.cli.package import package
import os
from pathlib import Path

# 创建 packages 文件夹
Path("./packages").mkdir(parents=True, exist_ok=True)

# 设置相对路径到项目根目录（当前 notebook 所在目录的上一级或你项目目录）
project_root = Path(".").resolve()  # 当前目录，也可以改成 Path("..").resolve() 如果 notebook 在子目录
os.environ["PYTHONPATH"] = f"{project_root}:{os.environ.get('PYTHONPATH','')}"

input_dir = Path("./models/spacy_lv/model-no-lemma")
output_dir = Path("./packages")

package(
    input_dir=input_dir,
    output_dir=output_dir,
    name="spacy_without_lemma",
    version="1.0.0",
    force=True
)


print("✅ 打包完成，可以在 packages 文件夹里找到")


/home/jesse/Projects/myenvs/spacy_lv/bin/python: No module named build


running sdist
running egg_info
creating lv_spacy_without_lemma.egg-info
writing lv_spacy_without_lemma.egg-info/PKG-INFO
writing dependency_links to lv_spacy_without_lemma.egg-info/dependency_links.txt
writing entry points to lv_spacy_without_lemma.egg-info/entry_points.txt
writing top-level names to lv_spacy_without_lemma.egg-info/top_level.txt
writing manifest file 'lv_spacy_without_lemma.egg-info/SOURCES.txt'
reading manifest file 'lv_spacy_without_lemma.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'lv_spacy_without_lemma.egg-info/SOURCES.txt'
running check
creating lv_spacy_without_lemma-1.0.0
creating lv_spacy_without_lemma-1.0.0/lv_spacy_without_lemma
creating lv_spacy_without_lemma-1.0.0/lv_spacy_without_lemma.egg-info
creating lv_spacy_without_lemma-1.0.0/lv_spacy_without_lemma/lv_spacy_without_lemma-1.0.0
creating lv_spacy_without_lemma-1.0.0/lv_spacy_without_lemma/lv_spacy_without_lemma-1.0.0/morphologizer
creating lv_spacy_without_lemma

In [ ]:
# ========================
# Cell 9: 安装并测试打包好的拉脱维亚语模型
# ========================
import subprocess
import spacy

package_path = "./packages/lv_spacy_without_lemma-1.0.0/dist/lv_spacy_without_lemma-1.0.0.tar.gz"

# 安装模型
subprocess.run(["pip", "install", package_path])

# 加载模型
nlp = spacy.load("lv_spacy_without_lemma")

# 查看 pipeline 组件
print("Pipeline组件：", nlp.pipe_names)
print("组件数量：", len(nlp.pipe_names))


In [ ]:
# ========================
# Cell 10: 测试模型，不显示 Lemma，并显示分句
# ========================
import spacy
import pandas as pd

nlp = spacy.load("lv_spacy_without_lemma")

text = """Rīga ir Latvijas galvaspilsēta un viens no galvenajiem rūpniecības, darījumu, kultūras, sporta un finanšu centriem Baltijas valstīs, kā arī nozīmīga ostas pilsēta. Ar 605 273 iedzīvotājiem (2024. gada dati) tā ir lielākā apdzīvotā vieta Latvijā. Tās robežās dzīvo aptuveni viena trešdaļa, bet Rīgas aglomerācijā — vairāk nekā puse visu Latvijas iedzīvotāju. Pilsētas teritorijas platība ir 307,17 km2. Rīgas vēsturiskais centrs ir iekļauts UNESCO Pasaules kultūras mantojuma sarakstā un ir ievērojams ar jūgendstila arhitektūru, kurai, pēc UNESCO viedokļa, nav līdzīgu pasaulē."""

doc = nlp(text)

# 先生成 token 表格
rows = []
for sent in doc.sents:
    for token in sent:
        rows.append({
            "Text": token.text,
            "POS": token.pos_,
            "Dependency": token.dep_,
            "Head": token.head.text
        })

df = pd.DataFrame(rows)

pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
display(df)  # 显示 token 表格

# 再单独显示分句
print("\n✅ 分句结果：")
for i, sent in enumerate(doc.sents, 1):
    print(f"Sentence {i}: {sent.text}")

